# Install and import libraries

In [1]:
%pip install pydot
%pip install tensorflow
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,
    METADATA_PATH,
)

CLASSES = {
    0: "resting",
    1: "palm up",
    2: "closed fist",
    3: "ok",
    4: "pointer finger",
    5: "peace",
    6: "shaa",
    7: "peace among worlds"
}

# Read the files in the data dir

In [3]:
# Read all of the files in the data folder
files_in_folder = Path(DATA_INPUT_PATH).glob("*.csv")

files = [x for x in files_in_folder]
print([file for file in files])

[WindowsPath('data/emg_gestures_data_20250128_220435.csv'), WindowsPath('data/emg_gestures_data_20250201_215436.csv'), WindowsPath('data/emg_gestures_data_20250202_172502.csv'), WindowsPath('data/emg_gestures_data_20250205_184712.csv'), WindowsPath('data/emg_gestures_data_20250205_212859.csv'), WindowsPath('data/emg_gestures_data_20250219_122322.csv'), WindowsPath('data/emg_gestures_data_20250226_122030.csv')]


## Convert .csv(s) to dataframes and concatenate

In [4]:
# Read the data from the files
dfs = []

for file in files:
    df = pd.read_csv(str(file))
    dfs.append(df)
        
# Convert the data to a DataFrame
df = pd.concat([x for x in dfs], axis=0)

columns = ["gesture_id", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8"]
df = df[columns]

print(df.head())

# Before removing duplicates
print(f"Shape of dataframe before removing duplicates {df.shape}")
# Remove duplicates
df = df.drop_duplicates()
print(f"Shape of dataframe after removing duplicates {df.shape}")

   gesture_id   s1   s2   s3   s4   s5   s6   s7  s8
0           0  183  264  188  118  119  115  178  94
1           0   91  203  149   71   60   66   86  49
2           0   73  229  158   80   53   48   69  40
3           0   60  230  154   76   45   43   55  35
4           0   53  264  204   81   41   37   47  32
Shape of dataframe before removing duplicates (117348, 9)
Shape of dataframe after removing duplicates (117348, 9)


In [5]:
# Parameters
window_size = 10  # Number of past samples to include

# Extract feature columns (excluding label, if it exists)
feature_columns = [col for col in df.columns if col != 'Label']
features = df[feature_columns].to_numpy()

# Convert features into a 3D array: [samples, features, time]
n_samples = len(features)
n_features = len(feature_columns)
reshaped_data = []

for i in range(n_samples):
    # Get the last `window_size` rows or fewer for each sample
    window = features[max(i - window_size + 1, 0):i + 1]
    
    # Pad the window if it has fewer than `window_size` rows
    if len(window) < window_size:
        padding = np.zeros((window_size - len(window), n_features))
        window = np.vstack((padding, window))
    
    reshaped_data.append(window)

# Convert list to 3D numpy array
reshaped_data = np.stack(reshaped_data, axis=0)
reshaped_data = reshaped_data.transpose(0, 2, 1)
df[feature_columns] = np.sqrt(np.mean(np.square(reshaped_data), axis=2))
    
print(df.shape)
print(df.head())

(117348, 9)
   gesture_id         s1          s2          s3         s4         s5  \
0         0.0  57.869681   83.484130   59.450820  37.314876  37.631104   
1         0.0  64.629715  105.311443   75.858421  43.548823  42.143801   
2         0.0  68.628711  127.806886   90.834465  50.363677  45.354162   
3         0.0  71.203230  147.053052  103.065513  55.804122  47.534198   
4         0.0  73.149163  169.098196  121.589884  61.401954  49.270681   

          s6         s7         s8  
0  36.366193  56.288542  29.725410  
1  41.929703  62.513998  33.521635  
2  44.592600  66.212537  35.828759  
3  46.619738  68.458747  37.499333  
4  48.065580  70.053551  38.840700  


## Scale and clean the data, then train the model

In [ ]:
X = df.drop(columns=['gesture_id'])
y = df['gesture_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for Conv1D
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# # filter for valid classes because the data is not clean
# valid_classes = [0, 2, 3]
# mask_train = y_train.isin(valid_classes)
# mask_test = y_test.isin(valid_classes)

# # apply the mask to the training and testing data
# X_train_filtered = X_train_scaled[mask_train]
# y_train_filtered = y_train[mask_train]

# X_test_filtered = X_test_scaled[mask_test]
# y_test_filtered = y_test[mask_test]

# one hot encode the target data
y_train_categorical = to_categorical(y_train, num_classes=8)
y_test_categorical = to_categorical(y_test, num_classes=8)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(8, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Compile the model with categorical crossentropy for multi-class classification
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

# Train the model using the filtered training data
model.fit(
    X_train_scaled, 
    y_train_categorical, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2, 
    callbacks=[early_stopping]
)

# Evaluate the model on the filtered test set
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_categorical)

print(f"Test accuracy: {test_acc}")

c:\Users\simpl\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8303 - loss: 0.5409 - val_accuracy: 0.9723 - val_loss: 0.0976
Epoch 2/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9514 - loss: 0.1608 - val_accuracy: 0.9782 - val_loss: 0.0810
Epoch 3/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9617 - loss: 0.1271 - val_accuracy: 0.9802 - val_loss: 0.0718
Epoch 4/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9654 - loss: 0.1190 - val_accuracy: 0.9820 - val_loss: 0.0655
Epoch 5/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9684 - loss: 0.1094 - val_accuracy: 0.9834 - val_loss: 0.0577
Epoch 6/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9709 - loss: 0.0996 - val_accuracy: 0.9846 - val_loss: 0.0563
Epoch 7/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9701 - loss: 0.0999 - val_accuracy: 0.9851 - val_loss: 0.0524
Epoch 8/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9717 - loss: 0

In [ ]:
# Save the model
model.save(f"model/{MODEL_PATH}")

# Save the scaler and the column names to a pickle file
with open(f"model/{METADATA_PATH}", 'wb') as f:
    pickle.dump((scaler, X_train.columns), f)

In [ ]:
# thumbs-up then peace-sign
emg_data = [
    [160,245,126,32,28,25,26,99], # thumbs-up
    [67,197,559,104,41,82,257,169], # peace-sign
    [205,440,165,40,27,83,229,226], # gun-fingers
    [416,434,134,71,36,48,102,103] # fist
]

for data in emg_data:
    emg_features_df = pd.DataFrame([data], columns=X_train.columns)

    emg_features_scaled = scaler.transform(emg_features_df)
    emg_features_reshaped = emg_features_scaled.reshape(1, -1)

    prediction = model.predict(emg_features_reshaped)
    predicted_class = np.argmax(prediction)

    print(f"Predicted class: {predicted_class} - {CLASSES[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Predicted class: 2 - closed fist
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted class: 7 - peace among worlds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class: 7 - peace among worlds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted class: 2 - closed fist
